<a href="https://colab.research.google.com/github/Isaezc/trabajo_3_analitica/blob/main/trabajo_3_Analitica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nombre de los integrantes:**

\

* **Integrante 1:** Juan José Monsalve Patiño \\
* **Integrante 2:** Pamela Escobar Marin \\
* **Integrante 3:** TuGatita09 \\
* **Integrante 4:** Juan Pablo Escobar Marin


**Definición del problema**




## Importacion de librerias


In [ ]:
from PIL import Image
from glob import glob
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import *
from pathlib import Path
from shutil import copyfile, rmtree
from skimage import feature, io
from skimage import io, transform
from skimage.feature import hog
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import shutil
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix, precision_recall_curve, auc, roc_curve
from tensorflow.keras import layers, models
from sklearn.metrics import ConfusionMatrixDisplay


# Análisis descriptivo y exploratorio

In [ ]:
without = 'trabajo_3_analitica/imagenes/without'
withh = 'imagenes/withh'
print('Cantidad de imagenes sin gafas:', len(os.listdir(without)))
print('Cantidad de imagenes con gafas:', len(os.listdir(withh)))

FileNotFoundError: ignored

In [ ]:
import os
print(os.getcwd())

/content


#Cargar y Preprocesar Imágenes

In [ ]:
dataset_dir = 'imagenes'
X = []
y = []
for label, category in enumerate(['withh','without']):
    category_dir=os.path.join(dataset_dir,category)
    for filename in os.listdir(category_dir):
        img_path=os.path.join(category_dir,filename)
        if os.path.isfile(img_path):
            img = io.imread(img_path)
            img = transform.resize(img,(64,64,3)).flatten()
            X.append(img)
            y.append(label)
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

FileNotFoundError: ignored

#Previsualizacion del dataset


In [ ]:
import matplotlib.pyplot as plt
fig,axes=plt.subplots(2,5,figsize=(12,6))
for label, category in enumerate(['withh','without']):
    category_dir=os.path.join(dataset_dir,category)
    image_files=os.listdir(category_dir)[:5]
    for i,filename in enumerate(image_files):
        img_path=os.path.join(category_dir, filename)
        img=io.imread(img_path)
        axes[label, i].imshow(img, cmap='gray')
        axes[label, i].set_title(category)
        axes[label, i].axis('off')
plt.show()

##Construir y Entrenar el Modelo

In [ ]:
model = models.Sequential([
    layers.Dense(4096,activation='relu',input_shape=(64*64*3,),kernel_initializer='glorot_uniform',bias_initializer='zeros'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

#Testing

In [ ]:
predictions = model.predict(X_test)
threshold = 0.5
predicted_classes = (predictions>threshold).astype(int)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, predicted_classes)
report = classification_report(y_test, predicted_classes)
print(f'Precisión: {accuracy}') #poner como una bolita que grafique el porcentaje,(que salga el resultado en porcentaje y redondear)

#Predicción

In [ ]:
img_path = '/content/imagenes/withh/choon_left_happy_sunglasses_4.pgm'
imagen_1 = io.imread(img_path)
imagen_1 = transform.resize(imagen_1,(64,64,3)).flatten()
imagen_1 = np.expand_dims(imagen_1,axis=0)
prediction = model.predict(imagen_1)
threshold = 0.5
predicted_class = (prediction > threshold).astype(int)
print(f'La probabilidad de que esta imagen contenga un rostro con gafas es de: {prediction[0][0]}')
print(f'Predicción (0: Sin Gafas, 1: Con Gafas): {predicted_class[0]}') # en vez de decir 0 o 1 imprimir un mensaje que diga "si contiene unas gafas" o "no contiene unas gafas"

In [ ]:
img_path2 = '/content/imagenes/withh/kawamura_up_sad_sunglasses_4.pgm'
imagen_2 = io.imread(img_path2)
imagen_2 = transform.resize(imagen_2, (64, 64, 3)).flatten()
imagen_2 = np.expand_dims(imagen_2, axis=0)
prediction2 = model.predict(imagen_2)
threshold = 0.5
predicted_class2 = (prediction > threshold).astype(int)
print(f'Probabilidad de tener gafas: {prediction2[0][0]}')
print(f'Predicción (0: Sin Gafas, 1: Con Gafas): {predicted_class2[0]}')

# Precisión, recall y F1-score

In [ ]:
#Precisión, Recall y F1-score
precision = precision_score(y_test, predicted_classes)
recall = recall_score(y_test, predicted_classes)
f1 = f1_score(y_test, predicted_classes)
print(f'Precision:{precision}')
print(f'Recall:{recall}')
print(f'F1-score:{f1}')
roc_auc = roc_auc_score(y_test, predictions)
print(f'Área bajo la curva: {roc_auc}')
## mostrar esto de una manera mas estetica, con imagenes o botones dibujaditos

# Matriz de Confusión

In [ ]:
cm = confusion_matrix(y_test, predicted_classes)
sns.set(font_scale=1.2)  # Ajusta el tamaño de la fuente
plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['witout', 'withh']).plot(cmap='Blues')## cambiar colores a la matriz
plt.title('Matriz de Confusión')
plt.show()

# Curva de Precisión-Recall

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)
area_under_curve = auc(recall, precision)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='darkorange', lw=2, label='Curva PR (area = {:.2f})'.format(area_under_curve)) #cambiar color y tamaños
plt.xlabel('Recall')
plt.ylabel('Precisión')
plt.title('Curva de Precisión y Recall')
plt.legend(loc='uppper right')
plt.show()

#Receiver operating characteristic curve

In [ ]:
fpr, tpr, _ = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkgreen', lw=2, label='Curva ROC (area = {:.2f})'.format(roc_auc))#cambiar color
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')#cambiar color
plt.xlabel('Relacion de Falsos Positivos')
plt.ylabel('Relacion de Verdaderos Positivos')
plt.title('Receiver operating characteristic curve')
plt.legend(loc='lower right')
plt.show()